In [95]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import  make_pipeline
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import re, nltk, gensim, spacy
import numpy as np
import mglearn
import konlpy
import csv
import pandas as pd
from konlpy.tag import *
import matplotlib.pyplot as plt
from pprint import pprint
from wordcloud import WordCloud, STOPWORDS
import pyLDAvis
import pyLDAvis.sklearn
import os
%matplotlib inline

In [2]:
!ls -al crawled/060920

total 5496428
drwxrwxr-x 2 ubuntu ubuntu         81 Jun 12 11:10 .
drwxrwxr-x 6 ubuntu ubuntu         62 Jun 13 03:53 ..
-rw-rw-r-- 1 ubuntu ubuntu  233601972 Jun 11 14:58 JEY_0609.txt
-rw-rw-r-- 1 ubuntu ubuntu 3646942501 Jun 12 05:56 Yongsoo_Lee_0609.txt
-rw-rw-r-- 1 ubuntu ubuntu 1747794684 Jun 12 07:38 YoonMiHyang0609.txt


In [3]:
data_file = os.getcwd() + '/crawled/060920/YoonMiHyang0609.txt'

In [4]:
if os.path.isfile(data_file):
    p_date = []
    media = []
    title = []
    news_text = []
    i = 0
    with open(data_file) as data_f:
        for line in data_f.readlines():
            if line.startswith('2020'):
                line_split = line.strip().split('\t')
                p_date.append(line_split[0])
                media.append(line_split[1])
                title.append(line_split[2])
                news_text.append(line_split[3])

In [33]:
news_data[:10000]

'2020.06.09.\t국민일보\t“지켜주지 못해 미안해”… 정의연 평화의집 소장 추모행사\t이나영 정의기억연대 이사장이 8일 서울 서대문구 신촌 세브란스병원 장례식장에 마련된 손 소장의 빈소에서 고인을 추모하고 있다. 추모의 밤 행사에 참석한 이들은 모두 검찰과 언론을 비판했다. 유튜브 캡처지난 6일 숨을 거둔 정의기억연대(정의연) ‘평화의 우리집’ 손모(60‧여) 소장을 기리는 행사가 9일 오후 빈소에서 열렸다. 정의연 활동가를 비롯한 참석자들은 손 소장이 생전 얼마나 헌신적으로 할머니들을 섬겼는지 기억하는 동시에 검찰과 언론을 향해 비판의 날을 세웠다.손 소장의 발인을 하루 앞둔 이날 오후 7시부터 서울 신촌세브란스병원 장레식장에서는 1시간여 동안 ‘추모의 밤’ 행사가 진행됐다. 추모행사 직전 고인을 기리기 위해 많은 조문객이 일시에 몰리면서 빈소가 가득 찼다. 유튜브 생중계에는 동시접속자가 200여명 가까이 몰렸다.참가자들은 2004년부터 삶을 마감하기 전까지 서울 마포구 ‘평화의 우리집’에서 일본군 위안부 피해자 할머니들을 돌본 손 소장의 삶이 헌신적이었다고 기억했다.김영순 한국여성단체연합 대표는 “(손 소장을) 이렇게 떠나보내는 현실을 견디기 힘들다. 누구나 할 수 있어보이지만 그렇지 않은 위대한 삶을 살았다”고 표현했다. 한국염 한국이주여성인권센터 대표도 손 소장과의 일화를 소개하며 “손 소장은 자기 자신을 포기한 사람이었다”면서 “할머니들을 돌보는 힘든 일에도 짜증 한 번 내지 않고 오히려 할머니들의 인정을 통해 힘을 얻는 사람이었다”고 회고했다.이들은 윤미향 더불어민주당 의원과 정의연에 대한 강제수사를 진행중인 검찰과 언론에 대한 불편함을 숨기지 않았다. 이나영 정의연 이사장은 “검찰의 과잉 수사와 언론의 취재 경쟁에 손 소장이 힘들어 하고 불안해했다”면서 “끝까지 쉼터에 계신 길원옥 할머니의 안위를 우선시하던 소장님을 지켜드리지 못해 정말 죄송하다”며 흐느꼈다. 한 센터장 역시 “손 소장이 쉼터를 쳐들어오는 검사들과 언론의 소나기를 버틸 거

In [50]:
news_text[0]

'이나영 정의기억연대 이사장이 8일 서울 서대문구 신촌 세브란스병원 장례식장에 마련된 손 소장의 빈소에서 고인을 추모하고 있다. 추모의 밤 행사에 참석한 이들은 모두 검찰과 언론을 비판했다. 유튜브 캡처지난 6일 숨을 거둔 정의기억연대(정의연) ‘평화의 우리집’ 손모(60‧여) 소장을 기리는 행사가 9일 오후 빈소에서 열렸다. 정의연 활동가를 비롯한 참석자들은 손 소장이 생전 얼마나 헌신적으로 할머니들을 섬겼는지 기억하는 동시에 검찰과 언론을 향해 비판의 날을 세웠다.손 소장의 발인을 하루 앞둔 이날 오후 7시부터 서울 신촌세브란스병원 장레식장에서는 1시간여 동안 ‘추모의 밤’ 행사가 진행됐다. 추모행사 직전 고인을 기리기 위해 많은 조문객이 일시에 몰리면서 빈소가 가득 찼다. 유튜브 생중계에는 동시접속자가 200여명 가까이 몰렸다.참가자들은 2004년부터 삶을 마감하기 전까지 서울 마포구 ‘평화의 우리집’에서 일본군 위안부 피해자 할머니들을 돌본 손 소장의 삶이 헌신적이었다고 기억했다.김영순 한국여성단체연합 대표는 “(손 소장을) 이렇게 떠나보내는 현실을 견디기 힘들다. 누구나 할 수 있어보이지만 그렇지 않은 위대한 삶을 살았다”고 표현했다. 한국염 한국이주여성인권센터 대표도 손 소장과의 일화를 소개하며 “손 소장은 자기 자신을 포기한 사람이었다”면서 “할머니들을 돌보는 힘든 일에도 짜증 한 번 내지 않고 오히려 할머니들의 인정을 통해 힘을 얻는 사람이었다”고 회고했다.이들은 윤미향 더불어민주당 의원과 정의연에 대한 강제수사를 진행중인 검찰과 언론에 대한 불편함을 숨기지 않았다. 이나영 정의연 이사장은 “검찰의 과잉 수사와 언론의 취재 경쟁에 손 소장이 힘들어 하고 불안해했다”면서 “끝까지 쉼터에 계신 길원옥 할머니의 안위를 우선시하던 소장님을 지켜드리지 못해 정말 죄송하다”며 흐느꼈다. 한 센터장 역시 “손 소장이 쉼터를 쳐들어오는 검사들과 언론의 소나기를 버틸 거라 생각했는데 미쳐 고통의 깊이를 생각하지 못했다”고 말했다.정의연은 추모행사에서 지난해 소천

In [7]:
#data_kkma = Kkma.nouns(news_text[0])

In [8]:
#data = Kkma.nouns(news_text)

In [15]:
#import mecab

In [16]:
from nltk.corpus import wordnet as wn

In [32]:
def hypernyms(word):
    current_node = wn.synsets(word)[0]
    yield current_node
    
    while True:
        try:
            current_node = current_node.hypernyms()[0]
            yield current_node
        except IndexError:
            break

In [33]:
def get_hypernyms(word):
    for wd in hypernyms(word):
        print(wd)

In [35]:
get_hypernyms('policeman')

Synset('policeman.n.01')
Synset('lawman.n.01')
Synset('defender.n.01')
Synset('preserver.n.03')
Synset('person.n.01')
Synset('causal_agent.n.01')
Synset('physical_entity.n.01')
Synset('entity.n.01')


In [184]:
def get_term_frequencies(document, word_dic = None, list = False):
    if word_dic is None:
        word_dic = {}
        
    if list == True:
        words = " ".join(document).split()
    else:
        words = document.split()
    
    for w in words:
        word_dic[w] = 1 + (0 if word_dic.get(w) is None else word_dic[w])
        
    return pd.Series(word_dic).sort_values(ascending=False)

In [53]:
tf_noun = get_term_frequencies(kkma.nouns(news_text[0]), list = True)
tf_morph = get_term_frequencies(kkma.morphs(news_text[0]), list = True)
tf = get_term_frequencies(news_text[0])

In [39]:
kkma = Kkma()

In [176]:
#tf_morph

In [177]:
#tf_noun

In [178]:
#tf

In [57]:
def get_document_frequency(documents):
    dicts = []
    vocab = set([])
    df = {}
    
    for d in documents:
        tf = get_term_frequencies(d)
        dicts += [tf]
        vocab = vocab | set (tf.keys())
        
    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
                
    return pd.Series(df).sort_values(ascending=False)

In [60]:
df = get_document_frequency(news_text[:1000])

In [179]:
#df

In [87]:
def get_tfidf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequencies(d, vocab)
        tfs += [get_term_frequencies(d)]
        
    df = get_document_frequency(docs)
    
    stats = []
    for word, freq in vocab.items():
        tfidfs = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            else:
                tfidfs += [0]
        
        stats.append((word, freq, *tfidfs, max(tfidfs)))
        
    return pd.DataFrame(stats, columns = ('word',
                                          'frequency',
                                          'doc1', 'doc2', 'doc3',
                                          'max')).sort_values('max', ascending=False)

In [94]:
get_tfidf([news_text[0],news_text[100], news_text[200]] )

,word,frequency,doc1,doc2,doc3,max
2,법안을,7,0.000000,7.690286,0.000000,7.690286
3,지역,5,0.000000,5.493061,0.000000,5.493061
5,법안,4,0.000000,4.394449,0.000000,4.394449
7,1호,4,0.000000,4.394449,0.000000,4.394449
9,통합당,4,0.000000,4.394449,0.000000,4.394449
31,지원을,3,0.000000,3.295837,0.000000,3.295837
13,관련,3,0.000000,3.295837,0.000000,3.295837
25,빈소를,3,3.295837,0.000000,0.000000,3.295837
26,부산,3,0.000000,3.295837,0.000000,3.295837
17,교수는,3,0.000000,0.000000,3.295837,3.295837


In [96]:
def get_tfidf_merged(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequencies(d, vocab)
        tfs += [get_term_frequencies(d)]
        
    df = get_document_frequency(docs)
    
    stats = []
    for word, freq in vocab.items():
        tfidfs = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            else:
                tfidfs += [0]
        
        stats.append((word, freq, np.argmax(tfidfs), max(tfidfs)))
        
    return pd.DataFrame(stats, columns = ('word',
                                          'frequency',
                                          'max_doc',
                                          'max')).sort_values('max', ascending=False)

In [181]:
get_tfidf_merged( news_text[:100] )

,word,frequency,max_doc,max
16,이동형>,95,80,56.918913
35,김운성>,75,80,44.935984
51,네.,65,80,38.944520
57,소녀상,60,80,35.948787
73,우리,55,80,32.953055
65,좀,59,80,26.487402
118,이런,40,80,23.965858
155,해서,35,80,20.970126
158,대해서,35,80,20.970126
34,것이,77,80,20.158396


In [100]:
#news_text[80]

In [139]:
def get_tf(docs, freq_size = 0):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequencies(d, vocab)
        tfs += [get_term_frequencies(d)]
        
    df = get_document_frequency(docs)
    
    vocab = {key:value for key, value in vocab.items() if value > freq_size} 
    stats = []
    for word, freq in vocab.items():
        tf_v = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_v += [tfs[idx][word]]
            else:
                tf_v += [0]
        
        stats.append((word, freq, *tf_v))
    
    cols = ['word', 'frequency'] + ['doc'+repr(i) for i in range(1, len(docs)+1)]
        
    return pd.DataFrame(stats, columns = (cols)).sort_values('frequency', ascending=False)

In [142]:
get_tf(news_text[:100])

,word,frequency,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,...,doc91,doc92,doc93,doc94,doc95,doc96,doc97,doc98,doc99,doc100
0,손,220,7,7,0,7,0,7,0,7,...,1,6,0,0,0,1,6,0,0,0
1,윤미향,181,1,1,0,1,0,1,0,1,...,0,1,3,5,7,0,1,3,5,7
2,의원은,164,1,1,0,1,0,1,0,1,...,8,2,2,0,1,8,2,2,0,1
3,소장의,157,3,3,0,3,0,3,0,3,...,0,4,4,0,3,0,4,4,0,3
4,이날,148,2,2,0,2,0,2,0,2,...,0,5,0,0,0,0,5,0,0,0
5,이,143,1,1,0,1,0,1,0,1,...,3,2,1,12,7,3,2,1,12,7
6,더불어민주당,128,1,1,0,1,0,1,0,1,...,1,3,3,0,0,1,3,3,0,0
7,한,124,2,2,0,2,0,2,0,2,...,2,0,0,6,2,2,0,0,6,2
8,대한,121,2,2,0,2,0,2,0,2,...,0,0,0,7,1,0,0,0,7,1
9,서울,119,3,3,0,3,0,3,0,3,...,0,2,0,0,0,0,2,0,0,0


In [124]:
docs = [news_text[0],news_text[100], news_text[200]]

In [145]:
from collections import defaultdict

In [171]:
def get_context_counts(docs, w_size=2):
    co_dict = defaultdict(int)
    
    for doc in docs:
        words = doc.split()
        
        for i, w in enumerate(words):
            for c in words[i - w_size: i + w_size]:
                if w != c:
                    co_dict[(w, c)] += 1
                    
    return pd.Series(co_dict).sort_values(ascending = False)

In [197]:
vocab = get_term_frequencies(news_text[:100], list = True)
co_dict = get_context_counts(news_text[:100], 2)

In [208]:
vocab = {k:v for k,v in vocab.items() if v > 30}

In [210]:
len(co_dict)

12848

In [213]:
def co_occurrence(co_dict, vocab):
    data = []
    i = 0
    
    for col in vocab:
        i += 1
        row = []
        for ind in vocab:
            try:
                count = co_dict[(col, ind)]
            except KeyError:
                count = 0
            row.append(count)
        data.append(row)
        print(i)
    
    return pd.DataFrame(data, index=vocab, columns=vocab)

In [214]:
co_occurrence(co_dict, vocab)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186


,손,윤미향,의원은,소장의,이날,이,더불어민주당,한,대한,서울,...,공천,지성호,때,국회의원의,무단,전재,이사장,의원과,더,따라
손,0,0,0,69,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
윤미향,0,0,15,0,0,0,68,0,0,0,...,0,0,5,0,0,0,11,11,0,0
의원은,0,53,0,0,29,33,38,0,0,0,...,0,22,0,0,0,0,0,0,0,0
소장의,69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
이날,0,0,29,0,0,0,29,0,0,0,...,0,0,0,0,0,0,0,0,0,0
이,0,4,33,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
더불어민주당,0,68,38,0,0,10,0,0,0,0,...,0,0,0,0,0,0,0,10,0,0
한,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
대한,20,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10,0,0
서울,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [161]:
#norm_dict['c'] += 1
co_dict['c'] += 1

In [164]:
co_dict

defaultdict(int, {})

In [156]:
d = defaultdict(int) 
   
L = [1, 2, 3, 4, 2, 4, 1, 2] 
   
# Iterate through the list 
# for keeping the count 
for i in L: 
       
    # The default value is 0 
    # so there is no need to  
    # enter the key first 
    d[i] += 1
       
print(d) 

defaultdict(<class 'int'>, {1: 2, 2: 3, 3: 1, 4: 2})


In [215]:
import torch

ModuleNotFoundError: No module named 'torch'

In [ ]:
torch.